In [4]:
from pyhive import hive
import pandas as pd


In [14]:
from urllib.parse import urlparse, unquote
import re
def url2domain(url):
    try:
        a = urlparse(unquote(url.strip()))
        if (a.scheme in ['http','https']):
            b = re.search("(?:www\.)?(.*)",a.netloc).group(1)
            if b is not None:
                return str(b).strip()
            else:
                return ''
        else:
            return ''
    except:
        return ''

In [5]:
# Читерим с подключением к хайву
conn = hive.Connection(host="bd-node1.newprolab.com", port=10000, username="sergey.korolev")

In [6]:
df = pd.read_sql("SELECT * FROM sergey_korolev.data", conn)
df['domains'] = df['data.url'].apply(url2domain)

In [603]:
df.head()

,data.uid,data.ts,data.url
0,298563565220,1.423711e+09,http%3A%2F%2Fsmotri.com%2Fbroadcast%2Flist%2F
1,298563565220,1.423711e+09,http%3A%2F%2Fsmotri.com%2Flive%2F7367249-vk%2F
2,119952932415,1.423711e+09,http%3A//www.echo.msk.ru/blog/nzlobin/1491452-...
3,119952932415,1.423711e+09,http%3A//www.echo.msk.ru/
4,300632579775,1.423711e+09,http%3A%2F%2Fwww.russianfood.com%2Frecipes%2Fr...


In [608]:
data_for_lab = {'Autouser': ['cars.ru', 'avto-russia.ru', 'bmwclub.ru'],
                'Cultureuser':  ['postnauka.ru', 'plantarium.ru', 'lensart.ru'],
                'Traveluser':  ['pass.rzd.ru', 'rzd.ru', 'vokrug.tv'],
                'Sickuser': ['apteka.ru', 'doctor.ufacity.info', 'womanhit.ru']}

In [677]:
df_with_categories = df.copy()

In [678]:
df_with_categories['categorie'] = None
for i in data_for_lab:
    df_with_categories.loc[df_with_categories['domains'].isin(data_for_lab[i]), 'categorie'] = i

In [681]:
df_with_categories.head()

,data.uid,data.ts,data.url,domains,categorie
0,298563565220,1.423711e+09,http%3A%2F%2Fsmotri.com%2Fbroadcast%2Flist%2F,smotri.com,None
1,298563565220,1.423711e+09,http%3A%2F%2Fsmotri.com%2Flive%2F7367249-vk%2F,smotri.com,None
2,119952932415,1.423711e+09,http%3A//www.echo.msk.ru/blog/nzlobin/1491452-...,echo.msk.ru,None
3,119952932415,1.423711e+09,http%3A//www.echo.msk.ru/,echo.msk.ru,None
4,300632579775,1.423711e+09,http%3A%2F%2Fwww.russianfood.com%2Frecipes%2Fr...,russianfood.com,None


In [682]:
df_with_categories.shape

(7207733, 5)

In [683]:
counted_data = df_with_categories.fillna(0).groupby(['data.uid', 'categorie','domains']).count()['data.url']
filtered_data = pd.DataFrame((counted_data >=10).astype(int))

In [684]:
filtered_data = filtered_data.reset_index()
filtered_data = filtered_data.drop_duplicates(['data.uid', 'categorie', 'data.url'])


In [685]:
filtered_data.head()

,data.uid,categorie,domains,data.url
0,100086165539,0,,1
2,100090111955,0,echo.msk.ru,1
4,100092973214,0,echo.msk.ru,1
6,100104407428,0,,1
8,100104407428,0,kurszdorovia.ru,0


In [686]:
result_data = pd.pivot_table(filtered_data, columns = 'categorie', index=['data.uid'], values='data.url', aggfunc='max')
result_data.head(30)

categorie,0,Autouser,Cultureuser,Sickuser,Traveluser
data.uid,,,,,
100086165539,1.0,NaN,NaN,NaN,NaN
100090111955,1.0,NaN,NaN,NaN,NaN
100092973214,1.0,NaN,NaN,NaN,NaN
100104407428,1.0,NaN,NaN,NaN,NaN
100116690612,1.0,NaN,NaN,NaN,NaN
10015223076,1.0,NaN,NaN,NaN,NaN
100155821422,1.0,NaN,NaN,NaN,NaN
100156142271,1.0,NaN,NaN,NaN,NaN
100178693790,1.0,NaN,NaN,NaN,NaN


In [687]:
result_data.reset_index(inplace=True)

In [688]:
my_columns = [i for i in data_for_lab]
my_columns.insert(0, 'data.uid')
final = result_data[my_columns ].fillna(0).astype('int')
final.sort_values('data.uid', inplace=True)
final['data.uid'] = final['data.uid'].apply(lambda x: str(x).strip())
final.to_csv('lab03_users.txt', sep='\t', index=False, header=False)

In [691]:
final.head(30)

categorie,data.uid,Autouser,Cultureuser,Traveluser,Sickuser
1371,1192,1,0,0,0
6860,1601191,0,0,1,0
8527,1688216,0,0,0,0
99177,8509607,0,0,0,0
710,11051384,0,0,0,0
1407,11961319,0,0,0,0
2613,13255383,0,0,0,0
10495,17800069,0,0,0,0
77867,30919606,0,0,0,0
97720,43175750,0,0,1,0


In [690]:
final['Autouser'].sum()

5209